Practice with Distance-based and Adversarial Domain Adaptation Methods
---
##### Author: [Sneh Pandya](https://snehjp2.github.io)

You've seen in the lectures how a variety of domain adaptation (DA) techniques can improve the generalization capabilities of neural networks (NNs), even when labeled data is only available in one domain. In this tutorial, you'll get hands-on experience applying DA to address a covariate shift scenario—where the input data distribution changes between training and deployment domains, but the underlying classification task remains the same.


<img src="https://snehjp2.github.io/images/confused.png" alt="confused" width="600"/>

We'll demonstrate this with a simple neural network trained on the [MNIST dataset](https://www.kaggle.com/datasets/hojjatk/mnist-dataset) (source domain) and evaluated on the [MNIST-M dataset](https://www.kaggle.com/datasets/aquibiqbal/mnistm) (target domain). While both datasets have the same label space (10 classes), the pixel distributions differ significantly, providing a natural testbed for domain adaptation techniques. In the interest of getting the point across simply, we will focus on a subset of both datasets, classifying 3 of the 10 classes (digits 1, 4, and 8). Our tutorial will use PyTorch, starting with the following imports:

In [ ]:
# torch related inputs
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from torchvision.datasets import MNIST, ImageFolder
import copy

# miscellaneous
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

import os
import random
import zipfile
import urllib

# geomloss provides distance measure that are torch/CUDA compatible. 
# Uncomment line below to install it, if you don't have it installed.

# !pip install -q geomloss

from geomloss import SamplesLoss

def set_all_seeds(seed=42):

    os.environ["PYTHONHASHSEED"] = str(seed)  # For Python's hash seed
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Optional: enforce deterministic algorithms where possible
    torch.use_deterministic_algorithms(True, warn_only=True)
set_all_seeds()


Problem Setup
---

- **Neural Network**:  
Let $f_\theta$ denote a neural network classifier with parameters $\theta$, which maps an input image $x \in \mathbb{R}^n$ to predicted class probabilities $\hat{y} = f_\theta(x) \in \mathbb{R}^K$. The network is decomposed into a feature extractor $\phi_\theta : \mathbb{R}^n \rightarrow \mathbb{R}^d$ and a classifier head $g_\theta : \mathbb{R}^d \rightarrow \mathbb{R}^K$, such that $f_\theta(x) = \text{softmax}(g_\theta(\phi_\theta(x)))$.  
We define the **latent representation** $z = \phi_\theta(x)$ as the output of the final hidden layer (before the logits). This representation (also called the *latent vector* or *latent space*) will be used for domain alignment.

- **Source domain dataset**:  
$\mathcal{D}_s = \{(x_s^{(i)}, y_s^{(i)})\}_{i=1}^{N_s}$,  
where $x_s^{(i)} \sim p_s(x)$ are grayscale digit images from MNIST, and $y_s^{(i)} \in \{1, 4, 8 \}$ are the corresponding digit labels.

- **Target domain dataset**:  
$\mathcal{D}_t = \{x_t^{(j)}\}_{j=1}^{N_t}$,  
where $x_t^{(j)} \sim p_t(x)$ are unlabeled images from MNIST-M.  
We assume the label distributions are aligned, i.e., $p_s(y|x) = p_t(y|x)$, but the input distributions differ: $p_s(x) \neq p_t(x)$.  
Our goal is to adapt $f_\theta$ using **only labeled source data** and **unlabeled target data**, so that it performs well on the target domain.

- **Classification loss**:  
On the source domain, we minimize the supervised cross-entropy loss between predicted and true labels:

$$
\mathcal{L}_{\text{CE}}(\theta) = -\frac{1}{N_s} \sum_{i=1}^{N_s} \log f_\theta^{(y_s^{(i)})}(x_s^{(i)}),
$$

where $f_\theta^{(k)}(x)$ denotes the predicted probability for class $k$, and $K = 3$ is the number of classes.

---
Let's start by defining our necessary ingredients, starting with the data. MNIST ($\mathcal{D}_s$) is readily available from `torchvision`, but we'll have to download MNIST-M ($\mathcal{D}_t$) from source. MNIST is single channel and MNISTM is three channel; we'll copy the single channel image across three channels so that the images can be loaded into the model properly. We will limit ourselves to a subset for the data so that you can train things rather quickly. 

Recall that images are typically normalized to be in the range [-1,1] for gradient stability during training. This is done via [z-score normalization](https://en.wikipedia.org/wiki/Standard_score). I've precalculated the means and standard deviations of the datasets for you. Feel free to check this yourself! This is an often not-discussed/overlooked aspect of training NNs, but data normalization is **immensely important**.

In [ ]:
mnist_mean = (0.1307,) * 3 ## found online
mnist_std = (0.3015,) * 3 ## found online

mnistm_mean = (0.4579, 0.4621, 0.4082) #I've precalculated these
mnistm_std = (0.1880, 0.1755, 0.1956) #I've precalculated these

In [ ]:
# Define target digits
target_digits = [1, 4, 8]

# Define transform
mnist_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3), 
    transforms.ToTensor(),
    transforms.Normalize(mnist_mean, mnist_std)
])

# Load full MNIST datasets
mnist_train_full = MNIST(root='./data', train=True, download=True, transform=mnist_transform)
mnist_test_full = MNIST(root='./data', train=False, download=True, transform=mnist_transform)

# Filter indices for only digits 1, 4, 8
def filter_indices(dataset, allowed_labels):
    return [i for i, (_, label) in enumerate(dataset) if label in allowed_labels]

filtered_train_indices = filter_indices(mnist_train_full, target_digits)
filtered_test_indices = filter_indices(mnist_test_full, target_digits)

# Create subsets for training and testing
train_subset_size = 5000
test_subset_size = 1000

train_indices = torch.tensor(filtered_train_indices)[torch.randperm(len(filtered_train_indices))[:train_subset_size]]
test_indices = torch.tensor(filtered_test_indices)[torch.randperm(len(filtered_test_indices))[:test_subset_size]]

mnist_train = Subset(mnist_train_full, train_indices)
mnist_test = Subset(mnist_test_full, test_indices)

# Define DataLoaders
source_train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
source_test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

In [ ]:
## Downloading MNIST-M and defining data loaders. MNIST-M labels are included but will not be used during training.

url = "https://github.com/mashaan14/MNIST-M/raw/main/MNIST-M.zip"
zip_path = "./MNIST-M.zip"
extract_path = "./data/mnist_m"

if not os.path.exists(zip_path):
    print("Downloading MNIST‑M dataset...")
    urllib.request.urlretrieve(url, zip_path)

if not os.path.exists(extract_path):
    print("Extracting MNIST‑M...")
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extract_path)

print("Files extracted to", extract_path)

In [ ]:
# Transform for MNIST-M
mnistm_transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize(mnistm_mean, mnistm_std)
])

# Load full MNIST-M dataset
mnistm_train_full = ImageFolder(root=os.path.join(extract_path, "MNIST-M/training"), transform=mnistm_transform)
mnistm_test_full = ImageFolder(root=os.path.join(extract_path, "MNIST-M/testing"), transform=mnistm_transform)

# Filter indices to only keep labels 1, 4, 8
def filter_indices(dataset, allowed_labels):
    return [i for i, (_, label) in enumerate(dataset) if label in allowed_labels]

filtered_train_indices = filter_indices(mnistm_train_full, target_digits)
filtered_test_indices = filter_indices(mnistm_test_full, target_digits)

# Create subsets for training and testing
train_indices = torch.tensor(filtered_train_indices)[torch.randperm(len(filtered_train_indices))[:train_subset_size]]
test_indices = torch.tensor(filtered_test_indices)[torch.randperm(len(filtered_test_indices))[:test_subset_size]]

# Create subset datasets
mnistm_train = Subset(mnistm_train_full, train_indices)
mnistm_test = Subset(mnistm_test_full, test_indices)

# Define DataLoaders
target_train_loader = DataLoader(mnistm_train, batch_size=64, shuffle=True)
target_test_loader = DataLoader(mnistm_test, batch_size=64, shuffle=False)

Now that the data is downloaded, lets just sanity check that the images look like what we would expect.

In [ ]:
# Collect one example per digit
def get_examples_by_label(dataset, target_digits):
    label_to_img = {}
    for img, label in dataset:

        label = int(label)
        if label in target_digits and label not in label_to_img:
            label_to_img[label] = img
        if len(label_to_img) == len(target_digits):
            break
    return label_to_img

def unnormalize(img, mean, std):
    img = img.clone()
    for c in range(img.shape[0]):
        img[c] = img[c] * std[c] + mean[c]
    return img

# Get one image per digit
mnist_digits = get_examples_by_label(mnist_train, target_digits)
mnistm_digits = get_examples_by_label(mnistm_train, target_digits)

# Plot the images
fig, axes = plt.subplots(2, len(target_digits), figsize=(12, 6))

for idx, digit in enumerate(target_digits):
    img_mnist = unnormalize(mnist_digits[digit], mnist_mean, mnist_std)
    axes[0, idx].imshow(img_mnist.permute(1, 2, 0).numpy())
    axes[0, idx].set_title(f"Digit {digit}", fontsize=10)
    axes[0, idx].axis("off")

    img_mnistm = unnormalize(mnistm_digits[digit], mnistm_mean, mnistm_std)
    axes[1, idx].imshow(img_mnistm.permute(1, 2, 0).numpy())
    axes[1, idx].set_title(f"Digit {digit}", fontsize=10)
    axes[1, idx].axis("off")

axes[0, 0].set_ylabel("MNIST", fontsize=12)
axes[1, 0].set_ylabel("MNIST-M", fontsize=12)
plt.tight_layout()
plt.show()


Images look good, now lets just check that the shapes are the same for training, and that the z-score normalization statistics look ok.

In [ ]:
# Get a sample image from each dataset
mnist_image = next(iter(DataLoader(mnist_train, batch_size=1)))[0]
mnistm_image = next(iter(DataLoader(mnistm_train, batch_size=1)))[0]

## TO DO ##
# Print shapes and statistics of the images
# Before printing, what do you expect the shapes and statistics to be?



Now let's define our model. We will write a simple CNN thats designed for classification, so that things can run quickly. There are a couple things to think carefully about here:

- The important difference with models used for DA training is that you are using the latent vector $z$ and the output logits. Write the forward pass to reflect this. That is, the full NN $f_\theta$ should output both the $\phi_\theta$ branch and $g_\theta$ branch.
- The efficacy of DA is primarily dependent on the expressivity of $\phi_\theta$. Do you expect this model to work well? Why or why not?
- Do we want the dimension of the latent vector to be small or large? Why or why not? 
- There are other NN architectural components (not included) that generally are thought to help generalization. What are some of these?

*Stretch Question:* You may have heard about symmetry-aware [equivariant NNs](https://arxiv.org/pdf/1602.07576). A NN like the one defined below is manifestly translation-equivariant, but many images in Nature contain higher-order symmetries (e.g., rotations, reflections). Would you expect an equivariant model to generalize better? Under what conditions would this happen, and under what conditions would this not? Hint: Think about how the conditional distribution of the labels transforms under symmetries.

Try to make a guess for how our model will perform before you proceed. Make note of your answers below:

In [ ]:
class CNN(nn.Module):
    def __init__(self, num_classes=3):
        super().__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2), 
        )

        self.flatten_dim = 128 * 7 * 7


        # Fully connected classifier
        ## TO DO ##
        # Write the linear layers to have the structure: Linear(128 * 7 * 7, 256) -> ReLU -> Dropout(0.5) -> Linear(256, 128) -> ReLU -> Linear(128, num_classes)
        self.classifier = None
        
    ## TO DO ##
    # Write the forward pass. Make sure to flatten the output of the convolutional layers before passing it to the classifier, and to return the latent z.
    def forward(self, x):
        pass

Lets proceed to train this model. We will train the CNN without domain adaptation on $\mathcal{D}_s$, and test it on both $\mathcal{D}_s$ and $\mathcal{D}_t$. This will serve as a performance baseline. Much of this training loop is standard. We will use `AdamW` optimizer with a learning rate of `1e-3`. Usually, one would split their training set further to include a validation set, and the best model will be saved according to that. For this simple example, we will just save the model according to the training set performance.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
CE_only_model = CNN().to(device)
label_map = {1: 0, 4: 1, 8: 2}

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(CE_only_model.parameters(), lr=1e-3)
best_model = None
best_loss = float('inf')
num_epochs = 3

# Training loop
for epoch in tqdm(range(num_epochs)):
    CE_only_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in source_train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        labels = torch.tensor([label_map[int(l)] for l in labels], device=device)

        ## TO DO ##
        # Forward pass through the model and compute loss. This is just two lines!

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if loss.item() < best_loss:
            best_loss = loss.item()
            best_model = copy.deepcopy(CE_only_model.state_dict())


        # Track training stats
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}]  Loss: {running_loss/len(source_train_loader):.4f}  Accuracy: {100*correct/total:.2f}%")

The loss went down and accuracy went up. All is well. For a more complex dataset, this would probably be trained for longer, and with some additional bells and whistles during training (validation, L2-regularization, gradient clipping, etc.) for better performance. But this will suffice for now. Now, lets test the model on both domains!

In [ ]:
def test_model(model, data_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images, labels = images.to(device), labels.to(device)
            labels = torch.tensor([label_map[int(l)] for l in labels], device=device)

            outputs, _ = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [ ]:
# Test on source domain (MNIST)
CE_only_model.load_state_dict(best_model)
source_accuracy = test_model(CE_only_model, source_test_loader, device)
print(f"Accuracy on source domain (MNIST): {source_accuracy:.2f}%")

# Test on target domain (MNIST-M)
target_accuracy = test_model(CE_only_model, target_test_loader, device)
print(f"Accuracy on target domain (MNIST-M): {target_accuracy:.2f}%")

We see that the model has a ~40% discrepancy between the source and target domain. Despite this pretty mild covariate shift, the model is failing horribly to generalize! Moreover, we saw from visualizing the images earlier that MNIST and MNIST-M dont look *too* different. This is important. **Typically trained NNs aren't learning dataset-invariant features**. Now, lets move to domain adaptation and see if we can shrink this performance gap between $\mathcal{D}_s$ and $\mathcal{D}_t$.

## Training with Distance-based Domain Adaptation

We use [`geomloss`](https://www.kernel-operations.io/geomloss/api/pytorch-api.html) to compute distributional distances between the latent representations of source and target examples. This library provides differentiable loss functions between samples or measures, making it well-suited for domain adaptation (DA). In this setup, the distance serves to **align the source and target latent distributions** without using target labels during backpropagation.

There are a variety of distances implemented in `geomloss`, such as Sinkhorn divergences and energy distances. In this example, we use the **energy distance**, which is equivalent to a Gaussian Maximum Mean Discrepancy (MMD) with a fixed bandwidth. You are encouraged to experiment with [other options](https://www.kernel-operations.io/geomloss/api/pytorch-api.html) and tune the DA loss strength $\lambda_{\text{MMD}}$ to explore their effects on adaptation performance.

We'll train our model for longer than before. Also, since we have access to our target labels, we’ll track target domain accuracy during training. In a general problem setting where target labels are unavailable, this wouldn't be possible — but for teaching purposes, it's a useful diagnostic. Importantly, **target labels are not used for backpropagation** and do not influence the model updates. We also save the model only according to training loss performance, not target domain accuracy.


#### Gaussian Maximum Mean Discrepancy (MMD)

Let $z_s \sim \mathbb{P}_s$ and $z_t \sim \mathbb{P}_t$ be the latent representations of source and target inputs respectively. The squared Gaussian MMD between these two distributions is defined as:

\begin{align}
\mathcal{L}_{\text{MMD}}(\theta) &\equiv \text{MMD}^2(\mathbb{P}_s, \mathbb{P}_t) \\
&= \mathbb{E}_{z, z' \sim \mathbb{P}_s} [k(z, z')] + \mathbb{E}_{\tilde{z}, \tilde{z}' \sim \mathbb{P}_t} [k(\tilde{z}, \tilde{z}')] - 2 \mathbb{E}_{z \sim \mathbb{P}_s,\, \tilde{z} \sim \mathbb{P}_t} [k(z, \tilde{z})],
\end{align}

where $k(z, \tilde{z}) = \exp\left( -\frac{\|z - \tilde{z}\|^2}{2\sigma^2} \right)$ is a Gaussian kernel with bandwidth $\sigma$. The full loss minimized during training is:

$$
\mathcal{L}_{\text{total}}(\theta) = \mathcal{L}_{\text{CE}}(\theta) + \lambda_{\text{MMD}} \cdot \mathcal{L}_{\text{MMD}}(\theta),
$$

where $\lambda_{\text{MMD}}$ controls the strength of the domain adaptation penalty. The goal is to correctly classify source examples while encouraging the latent feature distributions of source and target data to align.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
DA_model = CNN().to(device)

# Setup
num_epochs = 6
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(DA_model.parameters(), lr=1e-3)
lambda_mmd = 0.1
geomloss_fn = SamplesLoss("energy", p=2) # GeomLoss with Sinkhorn distance, p=2, blur ~ σ; tune as needed
best_model = None
best_loss = float('inf')

# Training loop
for epoch in tqdm(range(num_epochs)):
    DA_model.train()
    running_loss, correct_s, correct_t, total_s, total_t = 0.0, 0, 0, 0, 0
    running_DA, running_CE = 0.0, 0.0

    # Iterate over source and target data loaders
    for (source_imgs, source_labels), (target_imgs, target_labels) in zip(source_train_loader, target_train_loader):
        source_imgs, source_labels = source_imgs.to(device), source_labels.to(device)
        target_imgs, target_labels = target_imgs.to(device), target_labels.to(device)
        source_labels= torch.tensor([label_map[int(l)] for l in source_labels], device=device)
        target_labels = torch.tensor([label_map[int(l)] for l in target_labels], device=device)


        ## TO DO ##
        # Concatenate source and target images, forward through the model. Call `logits` the output of the model and `z` the latent representation.

        ## TO DO ##
        # Split logits and latent representations between the source and target domains. Call the source latent representation `z_s` and target latent representation `z_t`. Call the logits for source domain `logits_s` and for target domain `logits_t`.

        ## TO DO ##
        # Compute the losses. Use `criterion` for the cross-entropy loss and `geomloss_fn` for the GeomLoss. Combine them into `total_loss` with the MMD loss scaled by `lambda_mmd`.
        
        # Model saving
        if total_loss.item() < best_loss:
            best_loss = total_loss.item()
            best_model = copy.deepcopy(DA_model.state_dict())

        # Backward pass
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()

        # Track training stats
        running_loss += total_loss.item()
        running_CE += ce_loss.item()
        running_DA += mmd_loss.item()
        _, predicted_s = logits_s.max(1)
        total_s += source_labels.size(0)
        correct_s += predicted_s.eq(source_labels).sum().item()
        _, predicted_t = logits_t.max(1)
        total_t += target_labels.size(0)
        correct_t += predicted_t.eq(target_labels).sum().item()

    print(f"Epoch [{epoch+1}/{num_epochs}], CE Loss: {running_CE/len(source_train_loader):.4f}, DA Loss: {running_DA/len(source_train_loader):.4f} Source Acc: {100*correct_s/total_s:.2f}%, Target Acc: {100*correct_t/total_t:.2f}%")

In [ ]:
DA_model.load_state_dict(best_model)
# Test on source domain (MNIST)
source_accuracy = test_model(DA_model, source_test_loader, device)
print(f"Accuracy on source domain (MNIST): {source_accuracy:.2f}%")

# Test on target domain (MNIST-M)
target_accuracy = test_model(DA_model, target_test_loader, device)
print(f"Accuracy on target domain (MNIST-M): {target_accuracy:.2f}%")

DA works! This is a a pretty modest ~10% improvement over the basic NN for the target domain, and we also see similar performance on the source domain. This may not seem like a lot, and this is actually expected. Can you think why? Recall that the efficacy of DA relies on the *expressivity* of $\phi_\theta$: how well it could learn features. In a CNN, much of the feauture learning is done by convolutional layers; in our architecture we have four! The most expressive CNNs on the block, e.g. ResNetX, have $X \in \{18, 34, 50 \}$ convolutional layers with residual connections to help model convergence. In addition to this, there's several other axes that contribute to how well the model gets aligned:

- training configurations
- choice of distance kernel
- dataset normalization

Feel free to experiment on some or all of these axes, especially in making the CNN deeper and more expressive, and you should see the domain alignment increase.

---

Trying an Adversarial Approach
---

<img src="https://www.researchgate.net/publication/349424256/figure/fig2/AS:992763986333698@1613704787924/llustration-of-Domain-Adversarial-Neural-Network-DANN-framework-It-is-a-classic-and.png" alt="confused" width="800"/>


We've seen how to implement a distance-based DA approach; now, we turn to another family of domain adaptation (DA) methods: **adversarial approaches**.

In adversarial domain adaptation, we again rely on the feature extractor $\phi_{\theta_f}(x)$ to map both source and target images into a shared latent space $z = \phi_{\theta_f}(x) \in \mathbb{R}^d$. The goal is to learn representations that are discriminative for the source task yet domain-invariant, such that a classifier trained on the source domain generalizes to the target.

To achieve this, we introduce an auxiliary domain classifier $d_{\theta_d} : \mathbb{R}^d \rightarrow [0,1]$ that tries to distinguish whether a latent vector $z$ comes from the source or target domain. Here, we make explicit that the parameters of the domain classifier are separate, and what are optimized in total is $\theta = \{\theta_f, \theta_d\}$.

Let $\mathcal{D}_s = \{(x_s^{(i)}, y_s^{(i)})\}_{i=1}^{N_s}$ and $\mathcal{D}_t = \{x_t^{(j)}\}_{j=1}^{N_t}$ as before. During training, we optimize two competing objectives:

- **Source classification loss** (same as before):
  $$
  \mathcal{L}_{\text{CE}}(\theta) = -\frac{1}{N_s} \sum_{i=1}^{N_s} \log f_\theta^{(y_s^{(i)})}(x_s^{(i)}),
  $$

- **Domain adversarial loss**:
  $$
  \mathcal{L}_{\text{dom}}(\theta) = -\frac{1}{N_s} \sum_{i=1}^{N_s} \log d_{\theta_d}(\phi_{\theta_f}(x_s^{(i)}))
  - \frac{1}{N_t} \sum_{j=1}^{N_t} \log \left(1 - d_{\theta_d}(\phi_{\theta_f}(x_t^{(j)}))\right)
  $$

The domain classifier $d_{\theta_d}$ is trained to minimize $\mathcal{L}_{\text{dom}}$, correctly distinguishing source from target domains. In contrast, the feature extractor $\phi_{\theta_f}$ is trained to maximize this loss (via a gradient reversal layer), thereby encouraging domain-invariant features. Simultaneously, the source classifier is trained to minimize $\mathcal{L}_{\text{CE}}$ and distinguish between the classes. This leads to domain-invariant features, because the model must be good at classifying the source domain but must fail at distinguishing features between domains.

This adversarial interaction is implemented using a gradient reversal layer (GRL), which multiplies the gradient from $\mathcal{L}_{\text{dom}}$ by $-1$ before backpropagating through $\phi_{\theta_f}$.

The total loss function is:
$$
\mathcal{L}_{\text{total}}(\theta) = \mathcal{L}_{\text{CE}}(\theta) - \lambda \cdot \mathcal{L}_{\text{dom}}(\theta),
$$
where $\lambda$ balances the classification and domain alignment objectives.

In the next section, we'll implement this framework using the Domain-Adversarial Neural Network (DANN) and compare it directly against MMD-based adaptation.


In [ ]:
from torch.autograd import Function

# Gradient Reversal Layer (GRL) implementation
class GradientReversal(Function):
    @staticmethod
    def forward(ctx, x, lambda_grl):
        ctx.lambda_grl = lambda_grl
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_grl * grad_output, None

def grad_reverse(x, lambda_grl=1.0):
    return GradientReversal.apply(x, lambda_grl)

# Domain Classifier implementation
class DomainClassifier(nn.Module):
    def __init__(self, in_dim):
        super(DomainClassifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 1)  # Binary output
        )
        self.project = nn.Linear(6272, 128)

    def forward(self, z, lambda_grl=1.0):
        z_rev = grad_reverse(z, lambda_grl)
        z_proj = self.project(z_rev)
        return self.net(z_proj)

In [ ]:
# Setup
adversarial_model = CNN().to(device)
domain_classifier = DomainClassifier(in_dim=128).to(device)  # set z_dim to latent dimension of CNN
optimizer_DANN = optim.AdamW(
    list(adversarial_model.parameters()) + list(domain_classifier.parameters()), lr=1e-3
)
criterion_ce = nn.CrossEntropyLoss()
criterion_domain = nn.BCEWithLogitsLoss()
lambda_grl = 0.25
num_epochs = 6
best_model = None
best_loss = float('inf')

# Training loop
for epoch in range(num_epochs):
    adversarial_model.train()
    domain_classifier.train()

    running_loss, running_ce, running_da = 0.0, 0.0, 0.0
    correct_s, correct_t, total_s, total_t = 0, 0, 0, 0

    # Iterate over source and target data loaders
    for (x_s, y_s), (x_t, y_t) in zip(source_train_loader, target_train_loader):
        x_s, y_s = x_s.to(device), y_s.to(device)
        x_t, y_t = x_t.to(device), y_t.to(device)
        
        y_s = torch.tensor([label_map[int(l)] for l in y_s], device=device)
        y_t = torch.tensor([label_map[int(l)] for l in y_t], device=device)

        # Forward pass
        x = torch.cat([x_s, x_t], dim=0)
        domain_labels = torch.cat([
            torch.ones(x_s.size(0), 1),
            torch.zeros(x_t.size(0), 1)
        ], dim=0).to(device)

        ## TO DO ##
        # Forward pass through the adversarial model. Call `logits` the output of the model and `z` the latent representation.
        # Split logits and latent representations between the source and target domains. Call the source latent representation `z_s` and target latent representation `z_t`. Call the logits for source domain `logits_s` and for target domain `logits_t`.

        ## TO DO ##
        # Compute the losses. Use `criterion_ce` for the cross-entropy loss and `criterion_domain` for the domain classification loss. 
        # criterion domain should take logits from the domain classifier and domain_labels as inputs.
        # compute the total loss `loss_total` as the sum of the cross-entropy loss and the domain classification loss.
        
        if loss_total.item() < best_loss:
            best_loss = loss_total.item()
            best_model = copy.deepcopy(adversarial_model.state_dict())

        # Backward pass
        optimizer_DANN.zero_grad()
        loss_total.backward()
        optimizer_DANN.step()

        # Track training stats
        _, pred_s = logits_s.max(1)
        _, pred_t = logits_t.max(1)
        correct_s += pred_s.eq(y_s).sum().item()
        correct_t += pred_t.eq(y_t).sum().item()
        total_s += y_s.size(0)
        total_t += y_t.size(0)

        running_loss += loss_total.item()
        running_ce += loss_ce.item()
        running_da += loss_da.item()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"CE Loss: {running_ce/len(source_train_loader):.4f}, "
          f"DA Loss: {running_da/len(source_train_loader):.4f}, "
          f"Source Acc: {100 * correct_s / total_s:.2f}%, "
          f"Target Acc: {100 * correct_t / total_t:.2f}%")


In [ ]:
adversarial_model.load_state_dict(best_model)
# Test on source domain (MNIST)
source_acc = test_model(adversarial_model, source_test_loader, device)
# Test on target domain (MNIST-M)
target_acc = test_model(adversarial_model, target_test_loader, device)

print(f"Source Test Accuracy: {source_acc:.2f}%")
print(f"Target Test Accuracy: {target_acc:.2f}%")

We see that our adversarial approach was also succesful, though less performant than the distance-based approach. In general, one can find that adversarial approaches can *exceed* distance-based approaches for DA, though requiring much more hyperparameter tuning. We have done none of that, which can explain the discrepancy.

---

## Studying Isomaps

We have now trained three models: `CE_only_model`, `DA_model`, and `adversarial_model`, with varying performance on the target domain. These models used the same underlying CNN. It's now interesting to study how these models latent representations differ, and what that can tell us about the success and failure points of the respective training methods.

Recall that our latent space $z$ is high-dimensional. In this case, 128-d. To understand any relevant structure *as people*, we will have to project this down to 2D. Luckily, there are a variety of methods designed to do this.

One such method is [Isomaps](https://en.wikipedia.org/wiki/Isomap). The isomap method reduces dimensionality by preserving geodesic (manifold) distances between points. First, a neighborhood graph is built using $k$-nearest neighbors. Geodesic distances $D_{ij}$ are then estimated via shortest paths over this graph. Finally, classical MDS is applied to the resulting distance matrix. The embedding is given by the top two eigenvectors of:

$$
\mathbf{B} = -\frac{1}{2} \mathbf{H} D^2 \mathbf{H}, \quad \text{where } \mathbf{H} = \mathbf{I} - \frac{1}{N} \mathbf{1}\mathbf{1}^\top.
$$

This gives us a 2D projection that captures the global structure of the latent space.


In [ ]:
from sklearn.manifold import Isomap
from sklearn.preprocessing import StandardScaler


def extract_latents_with_labels(model, source_loader, target_loader, device):
    model.eval()
    z_all, y_all, domains = [], [], []

    with torch.no_grad():
        for imgs, labels in source_loader:
            imgs = imgs.to(device)
            _, z = model(imgs)
            z_all.append(z.cpu())
            y_all.extend([label_map[int(y)] for y in labels])

            domains.extend(["source"] * len(labels))

        for imgs, labels in target_loader:
            imgs = imgs.to(device)
            _, z = model(imgs)
            z_all.append(z.cpu())
            y_all.extend([label_map[int(y)] for y in labels])

            domains.extend(["target"] * len(labels))

    z_all = torch.cat(z_all, dim=0)
    z_all = torch.tensor(StandardScaler().fit_transform(z_all))
    return z_all, np.array(y_all), np.array(domains)

def plot_isomap_with_colors_and_markers(z_all, y_all, domains, title, xlim=None, ylim=None, subset=500, ax=None, legend=False):

    isomap = Isomap(n_components=2, n_neighbors=20)
    z_2d = isomap.fit_transform(z_all.numpy())

    indices = np.random.choice(len(z_all), size=subset, replace=False)
    z_2d = z_2d[indices]
    y_all = y_all[indices]
    domains = domains[indices]

    if ax is None:
        fig, ax = plt.subplots(figsize=(7, 5))

    cmap = plt.cm.get_cmap('tab10', 3)

    for digit in range(3):
        color = cmap(digit)
        for domain, marker in zip(["source", "target"], ["o", "^"]):
            idx = (y_all == digit) & (domains == domain)
            ax.scatter(
                z_2d[idx, 0], z_2d[idx, 1],
                label=f"{digit} ({'src' if domain == 'source' else 'tgt'})",
                c=[color], marker=marker, s=15, alpha=0.7
            )

    ax.set_title(title)
    ax.set_xlabel("Isomap dim 1")
    ax.set_ylabel("Isomap dim 2")
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)
    if legend:
        ax.legend(markerscale=1.5, fontsize=9, loc='center left', bbox_to_anchor=(1.02, 0.5))

    if ax is None:
        plt.tight_layout()
        plt.show()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# CE-only model
z_ce, y_ce, d_ce = extract_latents_with_labels(CE_only_model, source_test_loader, target_test_loader, device)
plot_isomap_with_colors_and_markers(z_ce, y_ce, d_ce, "Isomap: CE Only", ax=axes[0], legend=False)

# MMD-based model
z_da, y_da, d_da = extract_latents_with_labels(DA_model, source_test_loader, target_test_loader, device)
plot_isomap_with_colors_and_markers(z_da, y_da, d_da, "Isomap: MMD Domain Adaptation", ax=axes[1], legend=False)

# Adversarial model
z_adv, y_adv, d_adv = extract_latents_with_labels(adversarial_model, source_test_loader, target_test_loader, device)
plot_isomap_with_colors_and_markers(z_adv, y_adv, d_adv, "Isomap: Adversarial Domain Adaptation", ax=axes[2], legend=True)


We can see that there is considerable overlap in the CE-only model. We still do see a slight improvement in the latent spaces that include DA. And this, in fact, does make sense. We wouldn't expect perfect, nice clusters for each class as the target domain performance is still not perfectly aligned with the source. If you play around with things more and get the source domain performance on-par with the target domain, you should see some nice isomaps like the ones visualized in Figure 5 [here](https://arxiv.org/pdf/2501.14048v2). 

---
After finishing and if you want to play around with making these results better, to recap here are a few options for things you can do:

* Make an even deeper CNN model with the components that generally help model convergence/generalization (dropout, batch normalization, pooling).
* Test the existing model (and/or a deeper one) on the full MNIST and MNIST-M datasets with all digits. How does the performance do? How do the isomaps look?
* Consider adding L2-regularization, gradient clipping, and/or a learning rate scheduler to stabalize the training.
* install `escnn` and construct a dihedral group $D_N$ equivariant model. Study how well the model generalizes compared to the CNN. How does the latent space look for this model?
* Play around with different choices of MMD kernel inside `geomloss`. Consider implementing a superposition of Gaussian kernels using `kernel='gaussian'` with varying scales for $\sigma$. How does the model perform?


For a dataset like MNIST, some combination of all of these can get pretty high in accuracy (~77% for MMD and DANN). See the leaderboard [here](https://paperswithcode.com/sota/domain-adaptation-on-mnist-to-mnist-m). Thus concludes this tutorial. Happy coding!